In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os


In [2]:
N = 30
np.random.seed(0) #Code B
# A =  np.bool_(np.random.randint(2, size=N))
# B =  np.bool_(np.random.randint(2, size=N))
A =  np.bool_(np.random.randint(2, size=N))
B =  np.bool_(np.random.randint(2, size=N))
C =  np.bool_(np.random.randint(2, size=N))

customer = ['c'+str(i+16000) for i in range(90)]

Fraud = A & B #& C

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)

df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 'destination':np.random.choice(customer, size=N),\
                   'A':A, 'B':B, "Fraud":Fraud})
df.drop(df[(A==0)&(B==0)].index, inplace=True)
df['Fraud_chain--orig--destination']=False
df.drop(df[df['orig']==df['destination']].index, inplace=True)

In [3]:
def create_chain(df_):
    df=df_.copy()
    fraudsters = df[df['Fraud']==True]
    index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), \
                                how='inner', left_on='orig', right_on='destination')

    # df.loc[index_from_merge['index'],'Fraud']=True
    df.loc[index_from_merge['index'],'Fraud_chain--orig--destination']=True
    df['From--orig']=1
    df['To--destination']=1
    return df

In [4]:
df = create_chain(df)


### creating fact, positive and negative examples

In [5]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [6]:
%%script true
create_facts_and_examples(df_= df, target='Fraud', 
                          predicates=['A', 'B', 'From--orig', 'To--destination'], output_dir="examples/chain-fraud-erman")

In [7]:
%%script true
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud', 'From--orig', 'To--destination'], output_dir="examples/transitivity-fraud-rule")

## Create simple transitivity problem

In [8]:
pd.merge(df[['orig','destination']], df[['orig','destination']], \
                            how='inner', left_on=['orig', 'destination'], right_on=['destination','orig'])

,orig_x,destination_x,orig_y,destination_y


In [9]:
df['Fraud--orig--destination']=df['Fraud']
df['Transfer--orig--destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination', 'Transfer--orig--destination'], 
                          output_dir="examples/simple-transitivity-fraud-rule", filter_null_columns=False)

Fraud_chain--orig--destination 16086 16014 flag= True
Fraud_chain--orig--destination 16011 16067 flag= True
Fraud_chain--orig--destination 16032 16075 flag= True
Fraud_chain--orig--destination 16024 16033 flag= True
Fraud_chain--orig--destination 16013 16085 flag= True
Fraud_chain--orig--destination 16058 16066 flag= False
Fraud_chain--orig--destination 16065 16052 flag= False
Fraud_chain--orig--destination 16036 16067 flag= False
Fraud_chain--orig--destination 16043 16004 flag= False
Fraud_chain--orig--destination 16002 16011 flag= False
Fraud_chain--orig--destination 16051 16086 flag= False
Fraud_chain--orig--destination 16080 16077 flag= False
Fraud_chain--orig--destination 16054 16056 flag= False
Fraud_chain--orig--destination 16038 16024 flag= False
Fraud_chain--orig--destination 16019 16029 flag= False
Fraud_chain--orig--destination 16046 16021 flag= False
Fraud_chain--orig--destination 16042 16025 flag= False
Fraud_chain--orig--destination 16056 16080 flag= False
Fraud_chain--or

In [15]:
df[['orig','destination','Fraud','Fraud_chain--orig--destination']].loc[[4,8]]

,orig,destination,Fraud,Fraud_chain--orig--destination
4,16086,16014,True,True
8,16051,16086,True,False


In [11]:
df[df['Fraud']==True]

,orig,destination,A,B,Fraud,Fraud_chain--orig--destination,From--orig,To--destination,Fraud--orig--destination,Transfer--orig--destination
2,16036,16067,True,True,True,False,1,1,True,1
4,16086,16014,True,True,True,True,1,1,True,1
5,16043,16004,True,True,True,False,1,1,True,1
7,16002,16011,True,True,True,False,1,1,True,1
8,16051,16086,True,True,True,False,1,1,True,1
13,16038,16024,True,True,True,False,1,1,True,1
19,16077,16061,True,True,True,False,1,1,True,1
21,16024,16033,True,True,True,True,1,1,True,1
22,16002,16032,True,True,True,False,1,1,True,1
26,16072,16013,True,True,True,False,1,1,True,1


In [12]:
df[df['Fraud_chain--orig--destination']==True][['orig','destination', 'Fraud_chain--orig--destination']]

,orig,destination,Fraud_chain--orig--destination
4,16086,16014,True
6,16011,16067,True
10,16032,16075,True
21,16024,16033,True
24,16013,16085,True


In [13]:
%%script true
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination'], output_dir="examples/fraud-and-fraud-chain")